<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/C1_Regress_Linear_ScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn


In [ ]:
# Import Bibliotecas
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt


from sklearn                   import datasets, linear_model
from sklearn.metrics           import mean_squared_error, r2_score
from sklearn.preprocessing     import MinMaxScaler
from sklearn.neural_network    import MLPRegressor
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest

%matplotlib inline

In [ ]:
## Help Functions

# Supress scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Load data
df = pd.read_parquet('/content/all_bovespa_15to23.parquet')

In [ ]:
df

In [ ]:
# Capturando a ação desejada
acao = 'VALE3'
df_acao = df[df['sigla_acao'] == acao]

In [ ]:
df_acao

In [ ]:
df_acao.dtypes

In [ ]:
df_acao.tail()

In [ ]:
# Criando as médias moveis
df_acao['MM5d'] = df_acao.loc[: ,('preco_fech')].rolling(5).mean()
df_acao['MM21d'] = df_acao.loc[: ,('preco_fech')].rolling(21).mean()


In [ ]:
df_acao.head()

In [ ]:
# Empurrando para trás ps valores das ações
df_acao['preco_fech'] = df_acao['preco_fech'].shift(-1)

df_acao.head()
""" Faço isso para prever o dia seguinte usando 
as features do dia anterior"""

In [ ]:
# Eliminando os dados nulos
df_acao.dropna(inplace=True)
df_acao
# Elimino aqui 21 linhas

In [ ]:
# Verificando quantidade de linhas
qtd_linha = len(df_acao)

qtd_linha_treino = qtd_linha - 545 #(70%)

qtd_linha_teste = qtd_linha - 18 #(29%)

qtd_linha_valid = qtd_linha_treino - qtd_linha_teste

info = (
    f'Linhas treino = 0: {qtd_linha_treino}',
    f'Linhas teste = {qtd_linha_treino}:{qtd_linha_teste}',
    f'Linhas validação = {qtd_linha_teste}:{qtd_linha}'

)

info

In [ ]:
#Reindexando o df
df_acao = df_acao.reset_index(drop=True)
df_acao

In [ ]:
#Separando features e labels
features = df_acao.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fech'], 1)
labels = df_acao['preco_fech']


In [ ]:
# escolhendo as melhores features utilizando o KBest
features_list = ('preco_abert', 'qtd_negocios', 'vol_negocios', 'MM5d', 'MM21d')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()

print('')
print('Melhores features')
print(k_best_features_final)

In [ ]:
#Excluindo as features de baixo peso
features = df_acao.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fech', 'preco_abert', 'MM21d'], 1)

In [ ]:
# Normalizando os Dados usando MinMaxSacale
scaler = MinMaxScaler().fit(features)
features_scale = scaler.transform(features)

print('Features: ', features_scale.shape)
print(features_scale)

In [ ]:
# Separando em treino e test
X_train = features_scale[:qtd_linha_treino]
X_test = features_scale[qtd_linha_treino:qtd_linha_teste]

Y_train = labels[:qtd_linha_treino]
Y_test = labels[qtd_linha_treino:qtd_linha_teste]

print(len(X_train), len(Y_train))
print(len(X_test), len(Y_test))

# Aplicação dos modelos

In [ ]:
# Alincando modelo de regressão linear
lr = linear_model.LinearRegression()
lr.fit(X_train, Y_train)
pred = lr.predict(X_test)
cd = r2_score(Y_test, pred)

f'Coeficiente de determinação: {cd * 100:2f}'

"""Utilizando os dados de 2015 à 2023
obtivemos um Coeficiente de determinação igual à 96.942529"""

In [ ]:
# Utilizando uma rede Neural simples
rn = MLPRegressor(max_iter=2000)

rn.fit(X_train, Y_train)
pred = rn.predict(X_test)
cd = r2_score(Y_test, pred)

f'Coeficiente de determinação: {cd * 100:2f}'
"""Utilizando os dados de 2015 à 2023
obtivemos um Coeficiente de determinação igual à 90.622527"""

In [ ]:
# reforçando a RN acima com hiperparametros
rn = MLPRegressor()

parameter_space = {
    'hidden_layer_sizes': [(i,) for i in list(range(1, 21))],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

search = GridSearchCV(rn, parameter_space, n_jobs=-1, cv=5)

search.fit(X_train, Y_train)
clf = search.best_estimator_
pred = search.predict(X_test)

cd = search.score(X_test, Y_test)

f'Coeficiente de determinação: {cd * 100:2f}'

"""Utilizando os dados de 2015 à 2023
obtivemos um Coeficiente de determinação igual à 83.492816"""


In [ ]:

# Executando a previsao
previsao=features_scale[qtd_linha_teste:qtd_linha]

data_pregao_full=df_acao['data_pregao']
data_pregao=data_pregao_full[qtd_linha_teste:qtd_linha]

res_full=df_acao['preco_fech']
res=res_full[qtd_linha_teste:qtd_linha]

pred=lr.predict(previsao)

df=pd.DataFrame({'data_pregao': data_pregao, 'real': res, 'previsao': pred})

df['real'] = df['real'].shift(+1)

df.set_index('data_pregao', inplace=True)

print(df)

In [ ]:
# Visualização grafica
plt.figure(figsize=(25,10))
plt.title("Preço das ações")
plt.plot(df['real'], label='real', color='blue', marker='o')
plt.plot(df['previsao'], label='previsao', color='orange', marker='o')
plt.xlabel('Data pregão')
plt.ylabel('Preço fechamento')
leg = plt.legend()
